<a href="https://www.kaggle.com/code/dsptlp/bird-1?scriptVersionId=169450796" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture
!pip install timm

In [2]:
%%capture
!pip install boto3

In [3]:
import time
from tqdm import tqdm_notebook as tqdm
#import tqdm.notebook import tqdm
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import timm
import warnings

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

from torch.cuda.amp import autocast, GradScaler
warnings.filterwarnings('ignore')

from timm.scheduler.cosine_lr import CosineLRScheduler

import boto3
from botocore.exceptions import NoCredentialsError

In [4]:
ret = torch.cuda.is_available()
print(ret)
ret = torch.cuda.current_device()
print(ret)
ret = torch.cuda.device_count()
print(ret)
ret = torch.cuda.get_device_name(0)
print(ret)

True
0
1
Tesla P100-PCIE-16GB


In [5]:
ACCESS_KEY = 'X'
SECRET_KEY = 'X'

In [6]:
def upload_to_aws(local_file, bucket, s3_file):
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [7]:
def download_from_aws(s3_file, bucket):
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    try:
        s3.download_file(bucket, s3_file, s3_file)
        print("Download Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [8]:
saved_model_name = 'Model_Epoch=7_VAL=0.95845337_tf_efficientnet_b7_ns_LR=1.0035385385193203e-08.tar'
download_from_aws(saved_model_name, 'w251-homework3')

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
!ls -l ../input/midsw251birds

In [ ]:
!ls -l

In [ ]:
class args:
    lr = 0.0001
    epochs = 8
    batch_size = 40 
    num_workers = 5
    folds = 5

In [ ]:
alldf = pd.read_csv('../input/midsw251birds/train.csv')
# Split the training dataset into a training and a validation
valdf = alldf[::args.folds]
trndf = alldf[~alldf.filename.isin(valdf.filename)]
# Load our test data
tstdf = pd.read_csv('../input/midsw251birds/test.csv')
metadf = pd.read_csv('../input/midsw251birds/metadata.csv')
metadf = metadf.set_index('label')
print(f'File shapes -- train : {trndf.shape}, valid : {valdf.shape}, test : {tstdf.shape}')
trndf.head()

In [ ]:
imgnetmeans = [0.22363983, 0.18190407, 0.2523437 ]
imgnetstds = [0.32451536, 0.2956294,  0.31335256]
# Using albumentations, check some examples here : https://albumentations.readthedocs.io/en/latest/examples.html 
def trntransforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Transpose(p=0.5),
# ==================================================================================
        A.OneOf([IAAAdditiveGaussianNoise(),GaussNoise(),], p=0.2),
        A.OneOf([MotionBlur(p=0.2),MedianBlur(blur_limit=3, p=0.1),Blur(blur_limit=3, p=0.1),], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([OpticalDistortion(p=0.3),GridDistortion(p=0.1),IAAPiecewiseAffine(p=0.3),], p=0.2),
        A.OneOf([CLAHE(clip_limit=2),IAASharpen(),IAAEmboss(),RandomBrightnessContrast(),], p=0.3),
        A.HueSaturationValue(p=0.3),
# ==================================================================================        
        ToTensorV2(),
        ])

def tsttransforms():
    return A.Compose([
        ToTensorV2(),
    ])

class BirdDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.data = df
        self.img_dir = '../input/midsw251birds/'
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        fname = self.data.iloc[idx]['filename']
        img_path = f'{self.img_dir}/{fname}'
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image = image)['image']
        image = image.float() / 255.
        label = -1 if self.mode=='test' else self.data.iloc[idx]['label']
        
        return image, label

In [ ]:
# Define our dataset
trndataset = BirdDataset(trndf, 'train', trntransforms())
valdataset = BirdDataset(valdf, 'valid', tsttransforms())
tstdataset = BirdDataset(tstdf, 'test', tsttransforms())

In [ ]:
# Test the dataset
img, label = next(iter(trndataset))
species = metadf.loc[label]['name']
print(f'Species : {species}')
#Image.fromarray(img)
imgviz = (img * 255).transpose(0, 2).numpy().astype(np.uint8)
Image.fromarray(imgviz)

In [ ]:
loaderargs = {'num_workers' : args.num_workers, 'batch_size':args.batch_size, 'pin_memory': False, 'drop_last': False}
trnloader = DataLoader(trndataset, shuffle = True, **loaderargs)
valloader = DataLoader(valdataset, shuffle = False, **loaderargs)
tstloader = DataLoader(tstdataset, shuffle = False, **loaderargs)

In [ ]:
# creates efficientnet-b0 architecture
device = torch.device("cuda:0")
#device = torch.device("cpu")

#https://github.com/rwightman/pytorch-image-models/blob/master/results/results-imagenet.csv
#efficientnet_b2, tf_efficientnet_l2_ns, efficientnet_el, dm_nfnet_f6, tf_efficientnet_b7_ns, tf_efficientnet_b6_ns, tf_efficientnet_b5_ns
model_name = 'tf_efficientnet_b7_ns'
model = timm.create_model(model_name, pretrained = True)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# Loss function
criterion = torch.nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
num_epochs = args.epochs

In [ ]:

# ==================================================================================
n_epochs = args.epochs
n_warmup_epochs = 2
n_steps = len(trnloader)
print(len(trnloader))

scheduler = CosineLRScheduler(
            optimizer,
            t_initial= n_steps * n_epochs + 1,
            lr_min=0.0000001,
            warmup_lr_init=0.00001,
            warmup_t= n_steps * n_warmup_epochs + 1)
# ==================================================================================

In [ ]:
# ==================================================================================
#LOAD SAVE MODEL STATE
# ==================================================================================

if 1==1:

    loc = 'cuda:{}'.format('0')
    checkpoint = torch.load(saved_model_name, map_location=loc)
    args.start_epoch = checkpoint['epoch']            
    print(checkpoint['epoch'])    
    print(checkpoint['arch'])    
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
        
    n_epochs = args.epochs
    n_warmup_epochs = 1
    n_steps = len(trnloader)
    print(len(trnloader))

    scheduler = CosineLRScheduler(
                optimizer,
                t_initial= n_steps * n_epochs + 1,
                lr_min=0.00000001,
                warmup_lr_init=0.0000001,
                warmup_t= n_steps * n_warmup_epochs + 1)    

In [ ]:
#class args:
#    lr = 0.00001
#    epochs = 30
#    batch_size = 121
#    num_workers = 8
#    folds = 5

#optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
## Loss function
#criterion = torch.nn.CrossEntropyLoss()
#scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
#num_epochs = args.epochs

In [ ]:
# ==================================================================================
scaler = GradScaler()

# ==================================================================================

In [ ]:
def save_checkpoint(state, filename):
    torch.save(state, filename)

In [ ]:
since = time.time()
global_step = 0
lrls = []

for epoch in range(num_epochs):
    epoch = epoch 
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print(str(optimizer.param_groups[0]['lr']))
    
    # =======================ORIG==================
    #scheduler.step()
    # =======================ORIG==================
            
    model.train()
    running_loss = 0.0
    tk0 = tqdm(trnloader, total=int(len(trnloader)))
    for step, batch in enumerate(tk0):
        
        # =======================NEW==================
        scheduler.step(global_step)
        global_step+=1
        lrls.append(optimizer.param_groups[0]['lr'])         
        # =======================NEW==================
                
        inputs = batch[0].to(device, dtype=torch.float)
        labels = batch[1].to(device).long()
        optimizer.zero_grad()
        
        # =======================NEW==================
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        # =======================NEW==================
        
        #outputs = model(inputs)
        #loss = criterion(outputs, labels)
        #loss.backward()
        #optimizer.step()
        running_loss += loss.item()
        tk0.set_postfix(train_loss=(running_loss / (step+1)))
        
        # =======================NEW==================
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # =======================NEW==================
        
                        
    valpreds = []
    model.eval()
    running_loss = 0.0
    tkval = tqdm(valloader, total=int(len(valloader)))
    for step, batch in enumerate(tkval):
        inputs = batch[0].to(device, dtype=torch.float)
        labels = batch[1].to(device).long()
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        valpreds .append(outputs)
        running_loss += loss.item()
        tkval.set_postfix(valid_loss=(running_loss / (step+1)))
    preds = torch.cat(valpreds).argmax(1).detach().cpu().numpy()
    print(f'Valid accuracy {(valdf.label.values == preds).mean():.4f}')
    
    accuracy_valid = round((valdf.label.values == preds).mean(),8)
    learning_rate = str(optimizer.param_groups[0]['lr'])
    
    PATH = "Model_Epoch=" +str(epoch) + "_VAL=" +str(accuracy_valid) + "_" + model_name + "_LR=" + learning_rate + ".tar"                    
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': model_name,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, PATH)    
    upload_to_aws(PATH, 'w251-homework3', PATH)

In [ ]:
# Plot
ax = pd.Series(lrls).plot(logy=True, figsize = (10, 4))
for i in range(0,n_epochs*n_steps+1,n_steps): 
    ax.axvline(i, linewidth=0.2, color='r', linestyle='--')
    print(i)
ax.set_xlabel("steps")
ax.set_ylabel("LR (log scale)")

In [ ]:
# Submit 
tstpreds = []
tktst = tqdm(tstloader, total=int(len(tstloader)))
for step, batch in enumerate(tktst):
    inputs = batch[0].to(device, dtype=torch.float)
    with torch.no_grad():
        outputs = model(inputs)
        tstpreds.append(outputs)
predicted_labels = torch.cat(tstpreds).argmax(1).detach().cpu().numpy()
tstdf['label'] = predicted_labels

In [ ]:
tstdf.to_csv('submission.csv', index = False)

In [ ]:
#!kaggle competitions submit -c midsw251birds -f submission.csv -m "Test"